## Apiux & SII: Fuerza entre entidades tributaria relacionada a probabilidad de contaminacion, definicion en base a IVA y contaminacion de representantes.
## Henry Vega (henrry.vega@api-ux.com)
## Data scientist

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from pyspark_dist_explore import hist
import matplotlib.pyplot as plt
from pyspark.sql.types import StringType,TimestampType

In [2]:
spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.kerberos.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "16g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "2") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()
warnings.filterwarnings('ignore', category=DeprecationWarning)
sc=spark.sparkContext
sc.setLogLevel ('ERROR')
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


In [3]:
#Se seleccionan las transacciones tipo 33 o 34 de transacciones comerciales
df=spark.sql("select dhdr_rut_emisor,dhdr_rut_recep,dhdr_iva,dhdr_mnt_total,dhdr_fch_emis_int from dwbgdata.header_dte_consolidada_enc_sas_analitica where (dtdc_codigo ='33' or dtdc_codigo='34')")
#df = df.withColumn("dhdr_fch_emis_int", from_unixtime(unix_timestamp(col("dhdr_fch_emis_int").cast("string"),"yyyyMMdd"),"yyyy-MM-dd HH:mm:ss"))
#df = df.withColumn("dhdr_fch_emis_int",df["dhdr_fch_emis_int"].cast(StringType()))
df=df.withColumn("dhdr_fch_emis_int",to_timestamp(col("dhdr_fch_emis_int").cast("string"), "yyyyMMdd"))
df.schema
df.show()

Hive Session ID = fe4c7cbd-0631-4007-a64c-8beeda59935b


+--------------------+--------------------+--------+--------------+-------------------+
|     dhdr_rut_emisor|      dhdr_rut_recep|dhdr_iva|dhdr_mnt_total|  dhdr_fch_emis_int|
+--------------------+--------------------+--------+--------------+-------------------+
|gGDb8NLhg0ajP7iXj...|njLBzZlYNhszJSrZm...|       0|  600000.00000|2020-01-01 00:00:00|
|6lSkCdxgHAkBREKV4...|YOK83ahT5JGcfkjXw...|       0| 1514826.00000|2020-01-05 00:00:00|
|wpRW8we88bp+B19pL...|mNKzwe6Z5TAI7rkgK...|       0|  424658.00000|2020-01-01 00:00:00|
|iO3RuGmu9S8j/qceg...|VKwRcXW+co+Q+vis8...|    3808|   23848.00000|2020-01-01 00:00:00|
|XgWqrLqNpA6bnLsy3...|Ca57v1mq+nwoq4vXh...|       0|  867596.00000|2020-01-01 00:00:00|
|jnH5LqARxTrxMMS28...|5ZqOVNXrlcSHP1TC7...|       0| 6295073.00000|2020-01-05 00:00:00|
|blxcKetEUE9ndr55s...|0ZQfoiDHFbPlrwSKZ...|    7460|   46723.00000|2020-01-01 00:00:00|
|zPyf6GJ20YbSOk+rx...|qqQeXNp01G77elWga...|   16137|  101070.00000|2020-01-02 00:00:00|
|J8M5NnLsq8virJEoW...|iCprrtCnZP

In [4]:
df.createOrReplaceTempView("comercial")

In [5]:
df.schema

StructType(List(StructField(dhdr_rut_emisor,StringType,true),StructField(dhdr_rut_recep,StringType,true),StructField(dhdr_iva,LongType,true),StructField(dhdr_mnt_total,DecimalType(22,5),true),StructField(dhdr_fch_emis_int,TimestampType,true)))

In [6]:
spark.sql("select count(*) from comercial").show()

+----------+
|  count(1)|
+----------+
|1175169116|
+----------+



In [7]:
#Leemos la tabla de representantes
df2=spark.sql("select REPR_FECHA_INICIO_VO, CONT_RUT,CONT_RUT_REPRESENTANTE,REPR_FECHA_TERMINO_VO from dw.dw_trn_djr_representantes_e ")
df2.createOrReplaceTempView("representante")
spark.sql("select count(*) from representante").show()
df2.show()

+--------+
|count(1)|
+--------+
| 3849502|
+--------+



+--------------------+--------------------+----------------------+---------------------+
|REPR_FECHA_INICIO_VO|            CONT_RUT|CONT_RUT_REPRESENTANTE|REPR_FECHA_TERMINO_VO|
+--------------------+--------------------+----------------------+---------------------+
| 2005-05-17 04:00:00|IRKZe9Dtp9/TtUF9P...|  Ne3lrSJsBNsqa+S0t...|  2008-01-16 03:00:00|
| 2005-05-17 04:00:00|97WP/yx9sOY13b5iZ...|  T6wFV1XZNFGeXDXZb...|                 null|
| 2005-05-17 04:00:00|z8EM/NWZ6Ys0i8Wv7...|  +/+w8yGKIYU2WZKVQ...|                 null|
| 2005-04-12 04:00:00|+BurxAXG1gfMM+rjG...|  Muoj0+tUbW1puNtuN...|                 null|
| 2005-05-17 04:00:00|UKoHUHGKdfzo3S1ix...|  8eEdNIjgOw0oXxeZR...|                 null|
| 2005-05-17 04:00:00|/7Z6Rk1xOYeDFBS0N...|  oIT7pwHpTuyg/wI6R...|                 null|
| 2005-05-17 04:00:00|/7Z6Rk1xOYeDFBS0N...|  SScvqzZ4o0hpk7+3d...|  2019-08-27 04:00:00|
| 2005-05-17 04:00:00|DaH7L9WV1+qo+4RSl...|  E/m8XqCCoAv/HtpAs...|                 null|
| 2005-05-17 04:00:00

In [8]:
df2.schema

StructType(List(StructField(REPR_FECHA_INICIO_VO,TimestampType,true),StructField(CONT_RUT,StringType,true),StructField(CONT_RUT_REPRESENTANTE,StringType,true),StructField(REPR_FECHA_TERMINO_VO,TimestampType,true)))

In [9]:

# Unimos la tabla de relaciones comerciales con la tabla de representantes, se analizara el rut del receptor de la transaccion
a=spark.sql("select * from comercial left join representante on comercial.dhdr_rut_recep=representante.CONT_RUT ")
a.createOrReplaceTempView("tabla")
spark.sql(" select count(*) from tabla").show()

a.show()

+----------+
|  count(1)|
+----------+
|4587633237|
+----------+



+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+
|     dhdr_rut_emisor|      dhdr_rut_recep|dhdr_iva|dhdr_mnt_total|  dhdr_fch_emis_int|REPR_FECHA_INICIO_VO|            CONT_RUT|CONT_RUT_REPRESENTANTE|REPR_FECHA_TERMINO_VO|
+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+
|j0FHQ6UaKNQxOu96y...|+A3g1+/EJfrydhbuj...|    2126|   15000.00000|2022-04-12 00:00:00|                null|                null|                  null|                 null|
|DTBFv1VPUYkO4Y67X...|+MsITdbQlcZeyFqxd...|    1812|   20000.00000|2020-05-07 00:00:00| 2018-01-26 03:00:00|+MsITdbQlcZeyFqxd...|  RF83ogK+o8eg4QDgr...|                 null|
|GfR2x0Ls0uVIKj6/4...|+Qlzb+MF30dJxY1Ba...|    3671|   22990.00000|2022-03-14 00:00:00|                null|                n

In [10]:
#Seleccionamos solamente las relaciones donde podemos asociar la transaccion a un representante de esa entidad

In [11]:
b=spark.sql("select * from tabla where tabla.CONT_RUT is not null")
b.createOrReplaceTempView("tabla")
spark.sql(" select count(*) from tabla").show() # cantidad de entradas en la tabla
b.show()


+----------+
|  count(1)|
+----------+
|4179352818|
+----------+



+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+
|     dhdr_rut_emisor|      dhdr_rut_recep|dhdr_iva|dhdr_mnt_total|  dhdr_fch_emis_int|REPR_FECHA_INICIO_VO|            CONT_RUT|CONT_RUT_REPRESENTANTE|REPR_FECHA_TERMINO_VO|
+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+
|OfOv6+ieOcqmCBoyJ...|+/+IMFryGSJkidvlc...|     778|    4870.00000|2022-05-25 00:00:00| 2018-10-31 03:00:00|+/+IMFryGSJkidvlc...|  Mw0g+csAAGNGXoHc9...|  2022-11-16 03:00:00|
|OfOv6+ieOcqmCBoyJ...|+/+IMFryGSJkidvlc...|     778|    4870.00000|2022-05-25 00:00:00| 2022-11-16 03:00:00|+/+IMFryGSJkidvlc...|  TTj82VUvdHPsOutf5...|                 null|
|FynCjCaAj9E3eByLf...|+/+IMFryGSJkidvlc...|    3800|   23800.00000|2022-04-14 00:00:00| 2018-10-31 03:00:00|+/+IMFryGSJkidvlc

In [12]:
# Ahora se debe hacer el cruce de los contaminados
df3 = spark.read.options(header=True,inferSchema=True,delimiter=",").csv("/home/cdsw/data/processed/contaminados.csv")
df3.createOrReplaceTempView("contaminados")


In [13]:
# Se une la data anterior con la tabla contaminados
spark.sql("select * from tabla left join contaminados on tabla.CONT_RUT_REPRESENTANTE=contaminados.cont_rut").createOrReplaceTempView("tabla")

In [14]:
# Se ve la cantidad de entradas de esa tabla
spark.sql(" select count(*) from tabla").show()
spark.sql("select * from tabla").show()

+----------+
|  count(1)|
+----------+
|4179352818|
+----------+



+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+--------+-----+
|     dhdr_rut_emisor|      dhdr_rut_recep|dhdr_iva|dhdr_mnt_total|  dhdr_fch_emis_int|REPR_FECHA_INICIO_VO|            CONT_RUT|CONT_RUT_REPRESENTANTE|REPR_FECHA_TERMINO_VO|cont_rut|score|
+--------------------+--------------------+--------+--------------+-------------------+--------------------+--------------------+----------------------+---------------------+--------+-----+
|PVFjiIt69vWZC2XGL...|+/+IMFryGSJkidvlc...|    1884|   11800.00000|2022-03-22 00:00:00| 2018-10-31 03:00:00|+/+IMFryGSJkidvlc...|  Mw0g+csAAGNGXoHc9...|  2022-11-16 03:00:00|    null| null|
|PVFjiIt69vWZC2XGL...|+/+IMFryGSJkidvlc...|    1884|   11800.00000|2022-03-22 00:00:00| 2022-11-16 03:00:00|+/+IMFryGSJkidvlc...|  TTj82VUvdHPsOutf5...|                 null|    null| null|
|WS8UGIQ0bEY4+F6ZQ...|+/+IMFryGSJkidvlc...|   2683

In [15]:
# Se cruzan los valores donde las fechas de emision coincide con el periodo de algun representante de esa entidad
spark.sql("select dhdr_rut_emisor,dhdr_rut_recep,dhdr_iva,REPR_FECHA_INICIO_VO, dhdr_fch_emis_int,REPR_FECHA_TERMINO_VO,CONT_RUT_REPRESENTANTE,score from tabla where (dhdr_fch_emis_int>=REPR_FECHA_INICIO_VO) and (dhdr_fch_emis_int<=REPR_FECHA_TERMINO_VO or REPR_FECHA_TERMINO_VO is null) and dhdr_iva>0").createOrReplaceTempView("tabla")
spark.sql(" select count(*) from tabla").show()

+----------+
|  count(1)|
+----------+
|1710444509|
+----------+



In [16]:
spark.sql(" select * from tabla").show()

+--------------------+--------------------+--------+--------------------+-------------------+---------------------+----------------------+-----+
|     dhdr_rut_emisor|      dhdr_rut_recep|dhdr_iva|REPR_FECHA_INICIO_VO|  dhdr_fch_emis_int|REPR_FECHA_TERMINO_VO|CONT_RUT_REPRESENTANTE|score|
+--------------------+--------------------+--------+--------------------+-------------------+---------------------+----------------------+-----+
|PVFjiIt69vWZC2XGL...|+/+IMFryGSJkidvlc...|    2387| 2018-10-31 03:00:00|2022-04-11 00:00:00|  2022-11-16 03:00:00|  Mw0g+csAAGNGXoHc9...| null|
|WS8UGIQ0bEY4+F6ZQ...|+/+IMFryGSJkidvlc...|   26832| 2018-10-31 03:00:00|2021-06-02 00:00:00|  2022-11-16 03:00:00|  Mw0g+csAAGNGXoHc9...| null|
|P4hjpI0EdQaD5Qwrd...|+/+IMFryGSJkidvlc...|    5636| 2018-10-31 03:00:00|2021-06-15 00:00:00|  2022-11-16 03:00:00|  Mw0g+csAAGNGXoHc9...| null|
|PVFjiIt69vWZC2XGL...|+/+IMFryGSJkidvlc...|    4774| 2018-10-31 03:00:00|2022-04-11 00:00:00|  2022-11-16 03:00:00|  Mw0g+csAAGNGX

In [17]:

# Se toman dos tablas donde hay coincidencia con alguna alerta y otra con todas las entradas para obtener la fraccion de documentos contaminados en numero y en valor
spark.sql("select dhdr_rut_emisor,dhdr_rut_recep, count(*) as count1,sum(dhdr_iva) as sum1 from tabla where score is not null group by dhdr_rut_emisor,dhdr_rut_recep").createOrReplaceTempView("aux2")
spark.sql("select dhdr_rut_emisor, dhdr_rut_recep, count(*) as count2,sum(dhdr_iva) as sum2 from tabla group by dhdr_rut_emisor,dhdr_rut_recep").createOrReplaceTempView("aux1")

In [18]:
spark.sql("select * from aux1").show()

+--------------------+--------------------+------+------+
|     dhdr_rut_emisor|      dhdr_rut_recep|count2|  sum2|
+--------------------+--------------------+------+------+
|P4hjpI0EdQaD5Qwrd...|+/+IMFryGSJkidvlc...|     8| 45186|
|OfOv6+ieOcqmCBoyJ...|+/+IMFryGSJkidvlc...|     1|   778|
|Y5pbdrxQ5HDvyRGv2...|+/+IMFryGSJkidvlc...|     1|  3321|
|d1ud7RniwWMUFavFi...|+/+IMFryGSJkidvlc...|     1|   991|
|PVFjiIt69vWZC2XGL...|+/+IMFryGSJkidvlc...|     6| 41382|
|WS8UGIQ0bEY4+F6ZQ...|+/+IMFryGSJkidvlc...|     2| 48208|
|W+KgSjqyg0ruXJt6O...|+/+IMFryGSJkidvlc...|     2| 15966|
|mB1kyvShd16kHgeEd...|+/+IMFryGSJkidvlc...|     2|  8390|
|shRPyAsNITRHoPWgK...|+/+IMFryGSJkidvlc...|     2| 37621|
|tK7X+Q/ho2Fyh7clL...|+/+IMFryGSJkidvlc...|     1|  4306|
|fNbGIovmthwhGeUH+...|+/+IMFryGSJkidvlc...|     1|  6066|
|FynCjCaAj9E3eByLf...|+/+IMFryGSJkidvlc...|     1|  3800|
|dwElNqcCQQiFyI3ic...|+/+IMFryGSJkidvlc...|     1|  3670|
|UmKrt88grTKYXaHL/...|+/+IMFryGSJkidvlc...|     1| 20755|
|faeqzt85JrwoQ

In [19]:
spark.sql("select aux1.dhdr_rut_emisor as dhdr_rut_emisor, aux1.dhdr_rut_recep as dhdr_rut_recep, CASE WHEN count1 is null then 0 else count1 end as count1, CASE WHEN sum1 is null then 0 else sum1 end as sum1, count2, sum2 from aux1 left join aux2 on (aux1.dhdr_rut_emisor=aux2.dhdr_rut_emisor and aux1.dhdr_rut_recep=aux2.dhdr_rut_recep)").createOrReplaceTempView("tabla")
spark.sql("select * from tabla").show()

+--------------------+--------------------+------+----+------+--------+
|     dhdr_rut_emisor|      dhdr_rut_recep|count1|sum1|count2|    sum2|
+--------------------+--------------------+------+----+------+--------+
|+48A826qiPFGlOude...|+/40pDAlqnoiz6GKf...|     0|   0|     1|    4726|
|/64tWUHj2Ry4OLQsQ...|+2rzYPrtJPDS7ymKO...|     0|   0|     6|   45600|
|/8qbtwq80+Wh/t3S1...|++uB1C4MZTlgVpkTE...|     0|   0|     6|   57923|
|/Ez/EVybs70vYyM+z...|+/40pDAlqnoiz6GKf...|     0|   0|     1|    1921|
|1LuNfb/EedigHNhvO...|+/7DbRVkbRN/1wxN2...|     0|   0|     1|   22665|
|2IUJpCOCUc3oDgrfV...|+0IpN6iICKxe6gzEp...|     0|   0|     4|  252510|
|3EO9hWRn59P4/9DLB...|+2isYgNAQ5sgd+F1S...|     0|   0|     1|    5508|
|3FOfVSnoQS2hp3tbe...|+/JJJahnlkwlGco4I...|     0|   0|    38|  109919|
|3TRSFJYppOvPqehD+...|++S1yR/YEwaqUh6kQ...|     0|   0|     4| 1025620|
|4RvHrmkpG2fP4DQop...|+2uSl7r/Qbddr9hGJ...|     0|   0|     1|    2578|
|7RXlMqTEpHsde/zc3...|+/+b88HAubkiug11W...|     0|   0|     1|  

In [20]:
# Ahora se obtiene la proporcion es la fuerza entre las entidades, consideradas como la fraccion del iva contaminado en cantidad de documentos y monto de iva
spark.sql("select dhdr_rut_emisor,dhdr_rut_recep,count1/count2 as Fa, sum1/sum2 as Fb from tabla").createOrReplaceTempView("tabla")
spark.sql("SELECT *  from tabla").show() 

+--------------------+--------------------+---+---+
|     dhdr_rut_emisor|      dhdr_rut_recep| Fa| Fb|
+--------------------+--------------------+---+---+
|/64tWUHj2Ry4OLQsQ...|+2rzYPrtJPDS7ymKO...|0.0|0.0|
|1LuNfb/EedigHNhvO...|+/7DbRVkbRN/1wxN2...|0.0|0.0|
|1ML/TpSMSHZkR6uyS...|++xAo8llfd1Nuvsqi...|0.0|0.0|
|2IUJpCOCUc3oDgrfV...|+0IpN6iICKxe6gzEp...|0.0|0.0|
|3EO9hWRn59P4/9DLB...|+2isYgNAQ5sgd+F1S...|0.0|0.0|
|3FOfVSnoQS2hp3tbe...|+/JJJahnlkwlGco4I...|0.0|0.0|
|3TRSFJYppOvPqehD+...|++S1yR/YEwaqUh6kQ...|0.0|0.0|
|4RvHrmkpG2fP4DQop...|+2uSl7r/Qbddr9hGJ...|0.0|0.0|
|5sL7UjLBMhqYlBgF5...|++/UTl2mwL/J484yK...|0.0|0.0|
|8Pvp3yMUcDg/CauZj...|++2DK/e8a0qWOS7QA...|0.0|0.0|
|9C/M6hODy/wMyT6zR...|+00TjpDYYqU+a0ama...|0.0|0.0|
|A+2qlWuj7AjllMb1D...|+0C7u5pCjWkiOCSeZ...|0.0|0.0|
|A+lMVBFHRi8YlmQ8T...|++ZFDDW4EDPoAaMj8...|0.0|0.0|
|AW9RBCycz8gmeN8Px...|++x8iI+s1LR4Ubwke...|0.0|0.0|
|BEUv3VhE9jPXOQYEc...|++H9C/d7ihzov0uIY...|0.0|0.0|
|CDxQ1PM3F74tde60e...|++G/wrT25NOP1+TgZ...|0.0|0.0|
|DPG9tNC392a

In [21]:
# Promediamos las fracciones de iva contaminado para obtener una fuerza entre entidades
spark.sql("select dhdr_rut_emisor,dhdr_rut_recep, round(Fa/2+Fb/2,5) as Fuerza from tabla").createOrReplaceTempView("fuerza")

In [22]:
spark.sql("select * from fuerza where Fuerza>0").show()

+--------------------+--------------------+-------+
|     dhdr_rut_emisor|      dhdr_rut_recep| Fuerza|
+--------------------+--------------------+-------+
|++NLsjlAw+wNNGd+2...|G4XDNQmTxclvWTIci...|    0.5|
|++NLsjlAw+wNNGd+2...|j/aV9Toizx+1oac0G...|0.33333|
|+0SBLQbdcvtL6VPcw...|LYo+m51vqYGUYeXBx...|    1.0|
|+0yKx5GSw/Q5FoSrp...|HxGVIQOZZnJ1C57zz...|    0.2|
|+48A826qiPFGlOude...|ZvAsOkvX0nh79eZXP...|0.55556|
|+5mmEatCPHGP3VkKB...|Wz8VWDd0OWFPESA6E...|    1.0|
|+67fL4Fs9VqDibQd/...|xHuR8cmgrdhfl92rl...|0.88889|
|+6Ero2RCzMBYfHDNC...|788kcQ8vBx8XfZnJZ...|    0.5|
|+7O+UcfcjsNMAya4B...|MtMjNMiVlit5NhNDS...|    0.5|
|+7j69Vma7N8Z9VZfO...|yLUXbSNvA8Z5v+Y0V...|    0.2|
|+7xx5hIMfnDou08LO...|ZPCkeS4Zcs7nuzryc...|0.33333|
|+80M6qAvWwGPTaLtJ...|G9eDTGzw9Wah4cD16...|    1.0|
|+97NJ1GrQJwtF9fZJ...|fsi/Uxh68goETuKUy...|    1.0|
|+A5Hi+quGhDHPaCxH...|ySnRI/X6fCyGiKYdH...|0.33333|
|+B8YsURwxpT5lsGEW...|XYNSOZv0qavlrjHzT...|    1.0|
|+D+jHBPZmuCKpOauO...|/y4WgNbRcqzW2/YzI...|    1.0|
|+DI5e8Qj0bz

In [23]:
spark.sql("select dhdr_rut_emisor as emisor,dhdr_rut_recep as receptor, Fuerza as Fi from fuerza  where Fuerza>0 ").createOrReplaceTempView("fuerza")


In [24]:
#spark.sql("SELECT *  from fuerza order by Fi asc").show() 
#spark.sql("SELECT *  from fuerza order by Fi desc").show() 

In [25]:
representante=spark.sql("select *  from fuerza").toPandas()
representante.to_csv('/home/cdsw/data/processed/fuerza_representante.csv', index=False)